## YOLOv3 모델 (PyTorch)

* https://github.com/ultralytics/yolov3
* https://www.ultralytics.com

### 모델 다운로드

In [ ]:
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
%pip install -qr requirements.txt

In [ ]:
import yaml
import torch
from IPython.display import Image, clear_output

print(f"Torch:{torch.__version__}({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


In [ ]:
# 폴더 지울떄
# !rm -rf yolov3

### 추론(Inference)

`detect.py`: YOLOv3 모델을 이용한 추론

In [ ]:
!ls data/images

In [ ]:
Image(filename='data/images/bus.jpg', width=600)

In [ ]:
Image(filename='data/images/zidane.jpg', width=600)

In [ ]:
%pwd

In [ ]:
!ls

In [ ]:
!python detect.py --weights yolov3.pt --img 640 --conf 0.25 --source data/images/

In [ ]:
Image(filename='runs/detect/exp3/zidane.jpg', width=600)

In [ ]:
Image(filename='runs/detect/exp3/bus.jpg', width=600)

## YOLOv5 모델 (PyTorch)

* https://github.com/ultralytics/yolov5
* https://www.ultralytics.com

### 모델 다운로드

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
import yaml
import torch
from IPython.display import Image, clear_output

### 추론(Inference)

`detect.py`: YOLOv3 모델을 이용한 추론

In [ ]:
!ls data/images

In [ ]:
Image(filename='data/images/zidane.jpg', width=600)

In [ ]:
Image(filename='data/images/bus.jpg', width=600)

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/

In [ ]:
Image(filename='runs/detect/exp/bus.jpg', width=600)

In [ ]:
Image(filename='runs/detect/exp/zidane.jpg', width=600)

## 포트홀 탐지 모델

### 데이터셋 다운로드

* 포트홀 데이터셋: https://public.roboflow.com/object-detection/pothole

In [ ]:
%mkdir /content/yolov5/pothole

In [ ]:
%cd /content/yolov5/pothole

In [ ]:
!curl -L "https://public.roboflow.com/ds/L1EfCSgBAp?key=qaWO6QrJ4r" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
Image(filename='/content/yolov5/pothole/train/images/img-100_jpg.rf.233751cac3399aa80b5e58a2afbb753a.jpg', width=1000)

In [ ]:
Image(filename='/content/yolov5/pothole/test/images/img-105_jpg.rf.3fe9dff3d1631e79ecb480ff403bcb86.jpg', width=1000)

In [ ]:
from glob import glob

train_img_list = glob('/content/yolov5/pothole/train/images/*.jpg')
test_img_list = glob('/content/yolov5/pothole/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/pothole/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

In [ ]:
import yaml

with open('/content/yolov5/pothole/train.txt', 'w') as f:
  f.write('`n'.join(train_img_list) + '`n')

with open('/content/yolov5/pothole/test.txt', 'w') as f:
  f.write('`n'.join(test_img_list) + '`n')

with open('/content/yolov5/pothole/valid.txt', 'w') as f:
  f.write('`n'.join(valid_img_list) + '`n')

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
%cat /content/yolov5/pothole/data.yaml

In [ ]:
%pwd

In [ ]:
%%writetemplate /content/yolov5/pothole/data.yaml

train: ./pothole/train/images
test: ./pothole/test/images
val: ./pothole/valid/images

nc: 1
names: ['pothole']

In [ ]:
%cat /content/yolov5/pothole/data.yaml

### 모델 구성

In [ ]:
import yaml

with open('/content/yolov5/pothole/data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml


### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
import time as time
%time
%cd /content/yolov5/

!python train.py --img 640 --batch 32 --epochs 100 --data ./pothole/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pothole_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls /content/yolov5/runs/train/pothole_results/

In [ ]:
from IPython.display import Image, display

In [ ]:
Image(filename='/content/yolov5/runs/train/pothole_results/results.png', width=1000)

In [ ]:
Image(filename='/content/yolov5/runs/train/pothole_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='/content/yolov5/runs/train/pothole_results/val_batch0_labels.jpg', width=1000)

### 검증(Validation)

In [ ]:
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --iou 0.65 --task test

### 추론(Inference)

In [ ]:
%ls runs/train/pothole_results/weights

In [ ]:
!python detect.py --weights runs/train/pothole_results/weights/best.pt --img 640 --conf 0.4 --source ./pothole/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'))
display(Image(filename=image_name))

In [ ]:
Image(filename='/content/yolov5/runs/detect/exp2/img-105_jpg.rf.3fe9dff3d1631e79ecb480ff403bcb86.jpg', width=1000)

### 모델 내보내기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir /content/drive/MyDrive/pothole
%cp /content/yolov5/runs/train/pothole_results/weights/best.pt /content/drive/MyDrive/pothole

## 연기 탐지 모델

### 데이터셋 다운로드

* 연기 데이터셋: https://public.roboflow.com/object-detection/wildfire-smoke/

In [ ]:
%mkdir /content/yolov5/smoke
%cd /content/yolov5/smoke
!curl -L "https://public.roboflow.com/ds/DS0LGsrR8G?key=ztCXPkKVgt" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from glob import glob

train_img_list = glob('/content/yolov5/smoke/train/images/*.jpg')
test_img_list = glob('/content/yolov5/smoke/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/smoke/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

In [ ]:
import yaml

with open('/content/yolov5/smoke/train.txt', 'w') as f:
  f.write('`n'.join(train_img_list) + '`n')

with open('/content/yolov5/smoke/test.txt', 'w') as f:
  f.write('`n'.join(test_img_list) + '`n')

with open('/content/yolov5/smoke/valid.txt', 'w') as f:
  f.write('`n'.join(valid_img_list) + '`n')

In [ ]:
%cat /content/yolov5/smoke/data.yaml

In [ ]:
%%writetemplate /content/yolov5/smoke/data.yaml

train: ./smoke/train/images
test: ./smoke/test/images
val: ./smoke/valid/images

nc: 1
names: ['smoke']

### 모델 구성

In [ ]:
import yaml

with open('/content/yolov5/smoke/data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
import time as time
%time
%cd /content/yolov5/

!python train.py --img 640 --batch 32 --epochs 100 --data ./smoke/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name smoke_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls /content/yolov5/runs/train/smoke_results/

In [ ]:
Image(filename='/content/yolov5/runs/train/smoke_results/results.png', width=1000)

In [ ]:
Image(filename='/content/yolov5/runs/train/smoke_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='/content/yolov5/runs/train/smoke_results/val_batch0_labels.jpg', width=1000)

### 검증(Validation)

In [ ]:
!python val.py --weight runs/train/smoke_results/weights/best.pt --data ./smoke/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python val.py --weight runs/train/smoke_results/weights/best.pt --data ./smoke/data.yaml --img 640 --iou 0.65 --task test

### 추론(Inference)

In [ ]:
!python detect.py --weight runs/train/smoke_results/weights/best.pt --img 640 --conf 0.4 --source ./smoke/test/images

In [ ]:
import glob

image_name = random.choice(glob.glob('runs/detect/exp2/*.jpg'))
display(Image(filename=image_name))

### 모델 내보내기

In [ ]:
%mkdir /content/drive/MyDrive/smoke/
%cp /content/yolov5/runs/train/smoke_results/weights/best.pt /content/drive/MyDrive/smoke/

## 안전모 탐지 모델

### 데이터셋 다운로드

* 안전모 데이터셋: https://public.roboflow.com/object-detection/hard-hat-workers

In [ ]:
%mkdir /content/yolov5/hardhat
%cd /content/yolov5/hardhat
!curl -L "https://public.roboflow.com/ds/V1bmharPxr?key=2oFjxS9aEk" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from glob import glob

train_img_list = glob('/content/yolov5/hardhat/train/images/*.jpg')
test_img_list = glob('/content/yolov5/hardhat/test/images/*.jpg')

print(len(train_img_list), len(test_img_list))

In [ ]:
from sklearn.model_selection import train_test_split

test_img_list, val_img_list = train_test_split(test_img_list, test_size=0.5, random_state=777)
print(len(test_img_list), len(val_img_list))

In [ ]:
import yaml

with open('/content/yolov5/hardhat/train.txt', 'w') as f:
  f.write('`n'.join(train_img_list) + '`n')

with open('/content/yolov5/hardhat/test.txt', 'w') as f:
  f.write('`n'.join(test_img_list) + '`n')

with open('/content/yolov5/hardhat/valid.txt', 'w') as f:
  f.write('`n'.join(valid_img_list) + '`n')

In [ ]:
%cat /content/yolov5/hardhat/data.yaml

In [ ]:
%%writetemplate /content/yolov5/hardhat/data.yaml

train: ./hardhat/train/images
test: ./hardhat/test/images
val: ./hardhat/test/images
# 요기 보면, valid를 test 나누어 썼기 때문에 경로 조심해야 한다.
nc: 3
names: ['head', 'helmet', 'person']

### 모델 구성

In [ ]:
import yaml

with open('data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
%time
%cd /content/yolov5/

!python train.py --img 416 --batch 64 --epochs 50 --data ./hardhat/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name hardhat_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls runs/train/hardhat_results/

In [ ]:
Image(filename='runs/train/hardhat_results/results.png', width=1000)

In [ ]:
Image(filename='runs/train/hardhat_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='runs/train/hardhat_results/val_batch0_labels.jpg', width=1000)

### 검증(Validation)

In [ ]:
!python val.py --weight runs/train/hardhat_results/weights/best.pt --data ./hardhat/data.yaml --img 416 --iou 0.65 --half

In [ ]:
!python val.py --weight runs/train/hardhat_results/weights/best.pt --data ./hardhat/data.yaml --img 416 --task test

### 추론(Inference)

In [ ]:
!python detect.py --weight runs/train/hardhat_results/weights/best.pt --img 416 --conf 0.4 --source ./hardhat/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'))
display(Image(filename=image_name))

### 모델 내보내기

In [ ]:
%mkdir /content/drive/MyDrive/hardhat
%cp /content/yolov5/runs/train/hardhat_results/weights/best.pt /content/drive/MyDrive/hardhat

## 주차공간 탐지 모델

### 데이터셋 다운로드

* 주차공간 데이터셋: https://public.roboflow.com/object-detection/pklot

In [ ]:
%mkdir /content/yolov5/pklot
%cd /content/yolov5/pklot
!curl -L "https://public.roboflow.com/ds/uTB9TrsAgh?key=5yPC4NCEMf" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from glob import glob

train_img_list = glob('/content/yolov5/pklot/train/images/*.jpg')
test_img_list = glob('/content/yolov5/pklot/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/pklot/valid/images/*.jpg')

print(len(train_img_list), len(test_img_list), len(valid_img_list))

In [ ]:
import yaml

with open('/content/yolov5/pklot/train.txt', 'w') as f:
  f.write('`n'.join(train_img_list) + '`n')

with open('/content/yolov5/pklot/test.txt', 'w') as f:
  f.write('`n'.join(test_img_list) + '`n')

with open('/content/yolov5/pklot/valid.txt', 'w') as f:
  f.write('`n'.join(valid_img_list) + '`n')

In [ ]:
%cat /content/yolov5/pklot/data.yaml

In [ ]:
%%writetemplate /content/yolov5/pklot/data.yaml

train: ./pklot/train/images
test: ./pklot/test/images
val: ./pklot/valid/images

nc: 2
names: ['space-empty', 'space-occupied']

### 모델 구성

In [ ]:
import yaml

with open('data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
%time
%cd /content/yolov5/

!python train.py --img 640 --batch 32 --epochs 50 --data ./pklot/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pklot_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls runs/train/pklot_results/

In [ ]:
Image(filename='runs/train/pklot_results/results.png', width=1000)

In [ ]:
Image(filename='runs/train/pklot_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='runs/train/pklot_results/val_batch0_labels.jpg', width=1000)

### 검증(Validation)

In [ ]:
!python val.py --weight runs/train/pklot_results/weights/best.pt --data ./pklot/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python val.py --weight runs/train/pklot_results/weights/best.pt --data ./pklot/data.yaml --img 640 --task test

### 추론(Inference)

In [ ]:
!python detect.py --weight runs/train/pklot_results/weights/best.pt --img 640 --conf 0.4 --source ./pklot/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display
# 약간 지금 ram 부족한거 보니 여기까지인가 보다 2022.12.26 오후 3:04 분 곧 죽을거 같고, 만약 돌이가면 exp확인하고 돌리자 
image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp4/*.jpg'))
display(Image(filename=image_name))

### 모델 내보내기

In [ ]:
%mkdir /content/drive/MyDrive/pklot
%cp /content/yolov5/runs/train/pklot_results/weights/best.pt /content/drive/MyDrive/pklot